In [2]:
import re
import json
import spacy
import unicodedata
import pandas as pd
from datasets import load_dataset
from nltk.stem import SnowballStemmer
import openpyxl

C:\Users\Jiahe\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
ds_cleaned = load_dataset("json", data_files="wildchat_en_cleaned.jsonl", split="train")

def extract_prompt_response_pairs(conversation):
    """
    Yields (user_prompt, ai_response) pairs.
    Each pair consists of a user's message and the *next* assistant response.
    """
    pairs = []
    i = 0
    while i < len(conversation) - 1:
        user = conversation[i]
        assistant = conversation[i + 1]
        if user.get('role') == 'user' and assistant.get('role') == 'assistant':
            pairs.append({
                "prompt": user.get("content", ""),
                "response": assistant.get("content", ""),
                "prompt_clean": user.get("content_clean", ""),       
                "response_clean": assistant.get("content_clean", "") 
            })
        i += 1
    return pairs

all_pairs = []
for ex in ds_cleaned:
    for pair in extract_prompt_response_pairs(ex["conversation"]):
        pair["conversation_id"] = ex.get("conversation_id")
        pair["model"] = ex.get("model")
        all_pairs.append(pair)

with open("wildchat_prompt_response_pairs.jsonl", "w", encoding="utf-8") as f:
    for pair in all_pairs:
        f.write(json.dumps(pair, ensure_ascii=False) + "\n")


In [1]:
import pandas as pd
import re
import unicodedata
import spacy
from nltk.stem.snowball import SnowballStemmer

# Load Excel and clean column names
src_file = "Task Statements.xlsx"
df = pd.read_excel(src_file)
df.rename(columns=lambda c: c.strip(), inplace=True)

# Set relevant columns
task_col = "Task"
title_col = "Title"
print("Task column:", task_col, "example ->", df[task_col].iloc[0])
print("Title column:", title_col, "example ->", df[title_col].iloc[0])

# NLP Preparation
nlp = spacy.load("en_core_web_sm")
stemmer = SnowballStemmer("english")

def normalize(text: str) -> str:
    text = unicodedata.normalize("NFKD", str(text))
    text = "".join(c for c in text if not unicodedata.combining(c))
    text = text.lower()
    text = re.sub(r"https?://\S+|www\.\S+|\b[\w\.-]+@[\w\.-]+\.\w+\b", "", text)
    text = re.sub(r"[.!?]", " ", text)
    text = re.sub(r"\b(\d+)\b", r"NUM_\1", text)
    return re.sub(r"\s+", " ", text).strip()

def standardize(text: str) -> str:
    doc = nlp(normalize(text))
    tokens = [
        tok.lemma_.lower()
        for tok in doc
        if not tok.is_stop and not tok.is_punct and tok.lemma_.strip()
    ]
    return " ".join(tokens)

# Apply standardization
df["task_standardized"] = df[task_col].astype(str).apply(standardize)

# Select columns to save
out_cols = ["O*NET-SOC Code", title_col, task_col, "task_standardized"]
df[out_cols].to_csv("Task_Statements_standardized.csv", index=False, encoding="utf-8")

print(f"Saved to Task_Statements_standardized.csv  (total {len(df)} rows)")


Task column: Task example -> Direct or coordinate an organization's financial or budget activities to fund operations, maximize investments, or increase efficiency.
Title column: Title example -> Chief Executives
Saved to Task_Statements_standardized.csv  (total 18796 rows)
